<a href="https://colab.research.google.com/github/mk0653/untitled/blob/master/iryou_FT_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

In [2]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 15.0 MB/s 
     |████████████████████████████████| 895 kB 74.2 MB/s 
     |████████████████████████████████| 636 kB 65.5 MB/s 
     |████████████████████████████████| 3.3 MB 78.1 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install colorama

In [4]:
import math
import random
import time
import warnings

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as T
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

from transformers import AutoModel,AutoTokenizer,get_cosine_schedule_with_warmup, AutoConfig, AdamW

In [5]:
class Config:
    model_name = 'roberta-large'
    pretrained_model_path = '/content/drive/MyDrive/roberta_large'
    output_hidden_states = True
    epochs = 2
    evaluate_interval = 100
    batch_size = 8
    device = 'cuda'
    seed = 67
    max_len = 256
    lr = 1e-6 * 0.975
    wd = 0.01
    nfolds = 5
    num_labels =1
    eval_schedule = [(float('inf'), 320), (0.5, 160), (0.49, 80), (0.48, 40), (0.47, 20), (0, 0)]

In [6]:
DATA_DIR = "/content/drive/MyDrive/[NLP]医療論文仕分けコンペ/"
OUTPUT_DIR = "/content/drive/MyDrive/output/models/"

In [7]:
warnings.filterwarnings("ignore")


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:
def init_logger(log_file= "./train.log"):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()

In [10]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed = 471
seed_torch(seed)

In [11]:
train = pd.read_csv(DATA_DIR + "train.csv")
test = pd.read_csv(DATA_DIR + "test.csv")
sub = pd.read_csv(DATA_DIR + "sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]

In [12]:
border = len(train[train["judgement"] == 1]) / len(train["judgement"])
print(border)

0.023282372444280715


In [13]:
def get_train_data(train):

    # 交差検証 用の番号を振ります。
    Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for n, (train_index, val_index) in enumerate(Fold.split(train, train["judgement"])):
        train.loc[val_index, "fold"] = int(n)
    train["fold"] = train["fold"].astype(np.uint8)

    return train

In [14]:
def get_test_data(test):
    return test

In [15]:
train = get_train_data(train)

In [16]:
class BaseDataset(Dataset):
    def __init__(self, df, model_name, include_labels=True):
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.df = df
        self.include_labels = include_labels

        self.title = df["title"].tolist()
        self.encoded = tokenizer.batch_encode_plus(
            self.title,
            padding = 'max_length',            
            max_length = 256,
            truncation = True,
            return_attention_mask=True
        )
        
        if self.include_labels:
            self.labels = df["judgement"].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.encoded['input_ids'][idx])
        attention_mask = torch.tensor(self.encoded['attention_mask'][idx])

        if self.include_labels:
            label = torch.tensor(self.labels[idx]).float()
            return input_ids, attention_mask, label

        return input_ids, attention_mask

In [17]:
'''
import torch
import torch.nn as nn


class AttentionHead(nn.Module):
    def __init__(self, h_size, hidden_dim=512):
        super().__init__()
        self.W = nn.Linear(h_size, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        
    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

class CLRPModel(nn.Module):
    def __init__(self,transformer,config):
        super(CLRPModel,self).__init__()
        self.h_size = config.hidden_size
        self.transformer = transformer
        self.head = AttentionHead(self.h_size)
        self.linear = nn.Linear(self.h_size, 1)
              
    def forward(self, input_ids, attention_mask):
        transformer_out = self.transformer(input_ids, attention_mask)
        x = self.head(transformer_out.last_hidden_state)
        x = self.linear(x)
        return x
  '''

'\nimport torch\nimport torch.nn as nn\n\n\nclass AttentionHead(nn.Module):\n    def __init__(self, h_size, hidden_dim=512):\n        super().__init__()\n        self.W = nn.Linear(h_size, hidden_dim)\n        self.V = nn.Linear(hidden_dim, 1)\n        \n    def forward(self, features):\n        att = torch.tanh(self.W(features))\n        score = self.V(att)\n        attention_weights = torch.softmax(score, dim=1)\n        context_vector = attention_weights * features\n        context_vector = torch.sum(context_vector, dim=1)\n\n        return context_vector\n\nclass CLRPModel(nn.Module):\n    def __init__(self,transformer,config):\n        super(CLRPModel,self).__init__()\n        self.h_size = config.hidden_size\n        self.transformer = transformer\n        self.head = AttentionHead(self.h_size)\n        self.linear = nn.Linear(self.h_size, 1)\n              \n    def forward(self, input_ids, attention_mask):\n        transformer_out = self.transformer(input_ids, attention_mask)\n

In [18]:

class BaseModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()

        self.model = T.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        out = self.model(input_ids=input_ids, attention_mask=attention_mask)
        out = self.sigmoid(out.logits).squeeze()

        return out


In [19]:

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))


In [20]:
def train_fn(train_loader, model, criterion, optimizer, epoch, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to train mode
    model.train()

    for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        y_preds = model(input_ids, attention_mask)
        #print(y_preds)
        #print(labels)
        loss = criterion(y_preds, labels)

        # record loss
        losses.update(loss.item(), batch_size)
        loss.backward()

        optimizer.step()

        if step % 100 == 0 or step == (len(train_loader) - 1):
            print(
                f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )

    return losses.avg

In [21]:
def valid_fn(valid_loader, model, criterion, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds = []

    for step, (input_ids, attention_mask, labels) in enumerate(valid_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        # compute loss
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask)

        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)

        # record score
        preds.append(y_preds.to("cpu").numpy())

        if step % 100 == 0 or step == (len(valid_loader) - 1):
            print(
                f"EVAL: [{step}/{len(valid_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )

    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [22]:
def inference():
    predictions = []

    test_dataset = BaseDataset(test, "roberta-large", include_labels=False)
    test_loader = DataLoader(
        test_dataset, batch_size=8, shuffle=False, num_workers=4, pin_memory=True
    )

    for fold in range(5):
        LOGGER.info(f"========== model: roberta-large fold: {fold} inference ==========")
        #transformer =RobertaForSequenceClassification.from_pretrained(Config.pretrained_model_path, config=config)  
        #model = CLRPModel(transformer, config)
        model = BaseModel(Config.pretrained_model_path)
        model.to(device)
        optimizer = T.AdamW(model.parameters(), lr=1e-5)

        model.load_state_dict(torch.load(OUTPUT_DIR + f"roberta-large_fold{fold}_best.pth")["model"])
        model.eval()
        preds = []
        for i, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            with torch.no_grad():
                y_preds = model(input_ids, attention_mask)
            preds.append(y_preds.to("cpu").numpy())
        preds = np.concatenate(preds)
        predictions.append(preds)
    predictions = np.mean(predictions, axis=0)

    return predictions

In [23]:
def train_loop(train, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # Data Loader
    # ====================================================
    trn_idx = train[train["fold"] != fold].index
    val_idx = train[train["fold"] == fold].index

    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)

    train_dataset = BaseDataset(train_folds, "roberta-large")
    valid_dataset = BaseDataset(valid_folds, "roberta-large")

    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        drop_last=False,
    )
    config = AutoConfig.from_pretrained(Config.pretrained_model_path)
    config.update({
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
            }) 

    # ====================================================
    # Model
    # ====================================================
    #model = BaseModel("roberta-large")
    
    #transformer = AutoModel.from_pretrained(Config.pretrained_model_path, config=config)  
    #model = CLRPModel(transformer, config)
    model = BaseModel(Config.pretrained_model_path)
    model.to(device)   
    optimizer = T.AdamW(model.parameters(), lr=2e-5)

    criterion = nn.BCELoss()

    # ====================================================
    # Loop
    # ====================================================
    best_score = -1
    best_loss = np.inf

    for epoch in range(3):
        start_time = time.time()
        
        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        valid_labels = valid_folds["judgement"].values

        # scoring
        score = fbeta_score(valid_labels, np.where(preds < border, 0, 1), beta=7.0)

        elapsed = time.time() - start_time
        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
        )
        LOGGER.info(f"Epoch {epoch+1} - Score: {score}")

        if score > best_score:
            best_score = score
            LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
            torch.save(
                {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"roberta-large_fold{fold}_best.pth"
            )

    check_point = torch.load(OUTPUT_DIR + f"roberta-large_fold{fold}_best.pth")

    valid_folds["preds"] = check_point["preds"]

    return valid_folds

In [24]:
def get_result(result_df):
    preds = result_df["preds"].values
    labels = result_df["judgement"].values
    score = fbeta_score(labels, np.where(preds < border, 0, 1), beta=7.0)
    LOGGER.info(f"Score: {score:<.5f}")

In [25]:
def main():
    # Training
    oof_df = pd.DataFrame()
    for fold in range(5):
        _oof_df = train_loop(train, fold)
        oof_df = pd.concat([oof_df, _oof_df])
        LOGGER.info(f"========== fold: {fold} result ==========")
        get_result(_oof_df)
        
    # CV result
    LOGGER.info(f"========== CV ==========")
    get_result(oof_df)
    
    # Save OOF result
    oof_df.to_csv(OUTPUT_DIR + "oof_df.csv", index=False)

    # Inference
    predictions = inference()
    predictions = np.where(predictions < border, 0, 1)

    # submission
    sub["judgement"] = predictions
    sub.to_csv(OUTPUT_DIR + "submission.csv", index=False, header=False)

In [ ]:
if __name__ == "__main__":
    main()

========== fold: 0 training ==========


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Some weights of the model checkpoint at /content/drive/MyDrive/roberta_large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/roberta_large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.

Epoch: [1][0/2714] Elapsed 0m 0s (remain 34m 56s) Loss: 0.6799 
Epoch: [1][100/2714] Elapsed 0m 39s (remain 17m 1s) Loss: 0.1407 
Epoch: [1][200/2714] Elapsed 1m 18s (remain 16m 20s) Loss: 0.1235 
Epoch: [1][300/2714] Elapsed 1m 57s (remain 15m 40s) Loss: 0.1020 
Epoch: [1][400/2714] Elapsed 2m 36s (remain 15m 1s) Loss: 0.1022 
Epoch: [1][500/2714] Elapsed 3m 15s (remain 14m 22s) Loss: 0.1025 
